Dado que el entrenamiento de redes neuronales es una tarea  muy costosa, **se recomienda ejecutar el notebooks en [Google Colab](https://colab.research.google.com)**, por supuesto también se puede ejecutar en local.

Al entrar en [Google Colab](https://colab.research.google.com) bastará con hacer click en `upload` y subir este notebook. No olvide luego descargarlo en `File->Download .ipynb`

**El examen deberá ser entregado con las celdas ejecutadas, si alguna celda no está ejecutadas no se contará.**

El examen se divide en tres partes, con la puntuación que se indica a continuación. La puntuación máxima será 10.

    
- [Actividad 1: Redes Recurrentes](#actividad_1): 10 pts
    - [Cuestión 1](#3.1): 2.5 pt
    - [Cuestión 2](#3.2): 2.5 pt
    - [Cuestión 3](#3.3): 2.5 pts
    - [Cuestión 4](#3.4): 1.25 pts
    - [Cuestión 5](#3.5): 1.25 pts



In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

<a name='actividad_1'></a>
# Actividad 1: Redes Recurrentes


- [Cuestión 1](#3.1): 2.5 pt
- [Cuestión 2](#3.2): 2.5 pt
- [Cuestión 3](#3.3): 2.5 pts
- [Cuestión 4](#3.4): 1.25 pts
- [Cuestión 5](#3.5): 1.25 pts

Vamos a usar un dataset de las temperaturas mínimas diarias en Melbourne. La tarea será la de predecir la temperatura mínima en dos días. Puedes usar técnicas de series temporales vistas en otras asignaturas, pero no es necesario.


In [2]:
dataset_url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv'
data_dir = tf.keras.utils.get_file('daily-min-temperatures.csv', origin=dataset_url)

67921/67921 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
df = pd.read_csv(data_dir, parse_dates=['Date'])
df.head()

,Date,Temp
0,1981-01-01,20.7
1,1981-01-02,17.9
2,1981-01-03,18.8
3,1981-01-04,14.6
4,1981-01-05,15.8


In [4]:
temperatures = df['Temp'].values
print('number of samples:', len(temperatures))
train_data = temperatures[:3000]
test_data = temperatures[3000:]
print('number of train samples:', len(train_data))
print('number of test samples:', len(test_data))
print('firsts trainn samples:', train_data[:10])

number of samples: 3650
number of train samples: 3000
number of test samples: 650
firsts trainn samples: [20.7 17.9 18.8 14.6 15.8 15.8 15.8 17.4 21.8 20. ]


<a name='3.1'></a>
## Cuestión 1: Convierta `train_data` y `test_data`  en ventanas de tamaño 5, para predecir el valor en 2 días

En la nomenclatura de [Introduction_to_RNN_Time_Series.ipynb](https://github.com/ezponda/intro_deep_learning/blob/main/class/RNN/Introduction_to_RNN_Time_Series.ipynb)
```python
past, future = (5, 2)
```

Para las primeras 10 muestras de train_data `[20.7, 17.9, 18.8, 14.6, 15.8, 15.8, 15.8, 17.4, 21.8, 20. ]` el resultado debería ser:

```python
x[0] : [20.7, 17.9, 18.8, 14.6, 15.8] , y[0]: 15.8
x[1] : [17.9, 18.8, 14.6, 15.8, 15.8] , y[1]: 17.4
x[2] : [18.8, 14.6, 15.8, 15.8, 15.8] , y[2]: 21.8
x[3] : [14.6, 15.8, 15.8, 15.8, 17.4] , y[3]: 20.             
```

In [47]:
# windowing function
def create_windows_np(data, window_size, horizon, shuffle = False):
  x, y = [], []
  for i in range(len(data) - window_size - horizon + 1):
    x.append(data[i:i + window_size])
    y.append(data[i+window_size + horizon-1])
  x, y = np.array(x), np.array(y)

  if shuffle:
    indices = np.arange(len(x))
    np.random.shuffle(indices)
    x, y = x[indices], y[indices]
  return x, y

In [48]:
past, future = (5, 2)
X_train, y_train = create_windows_np(train_data, window_size = past, horizon = future)
X_test, y_test = create_windows_np(test_data, window_size = past, horizon = future)

<a name='3.2'></a>
## Cuestión 2: Cree un modelo recurrente de dos capas GRU para predecir con las ventanas de la cuestión anterior.


In [50]:
inputs = keras.layers.Input(shape=(past, 1), name = 'input')
layer1 = layers.GRU(units = 32, return_sequences = True, name = 'layer_1')(inputs)
layer2 = layers.GRU(units = 32, return_sequences = False, name = 'layer_2')(layer1)
dense = layers.Dense(1, name = 'output')(layer2)
model = keras.Model(inputs=inputs, outputs=dense)
model.compile(optimizer = keras.optimizers.Adam(), loss = 'mse')
model.summary()

Model: "functional_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input (InputLayer)                   │ (None, 5, 1)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ layer_1 (GRU)                        │ (None, 5, 32)               │           3,360 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ layer_2 (GRU)                        │ (None, 32)                  │           6,336 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output (Dense)                       │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 9,729 (38.00 KB)

 Trainable params: 9,729 (38.00 KB)

 Non-trainable params: 0 (0.00 B)

In [51]:
es_callback = keras.callbacks.EarlyStopping(
    monitor="val_loss", min_delta=0, patience=10)

history = model.fit(
    X_train, y_train,
    epochs=200,
    validation_split=0.2, shuffle=True, batch_size = 64, callbacks=[es_callback]
)

Epoch 1/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - loss: 101.5477 - val_loss: 55.3575
Epoch 2/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 36.3377 - val_loss: 32.6088
Epoch 3/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 23.7017 - val_loss: 24.9310
Epoch 4/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 19.0830 - val_loss: 21.2176
Epoch 5/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 17.8355 - val_loss: 19.1123
Epoch 6/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 17.2640 - val_loss: 17.6681
Epoch 7/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 15.2217 - val_loss: 15.9023
Epoch 8/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 13.6363 - val_loss: 14.0052
Epoch 9/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 11.9599 - val_loss: 12.2938
Epoch 10/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 10.8235 - val_loss: 11.3831
Epoch 11/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 10.0369 - val_loss: 10.5618
Epoch 12/200
38/38 ━━━━━━━━━━━━━━━━━━━━

In [52]:
results = model.evaluate(X_test, y_test, verbose=1)
print('Test Loss: {}'.format(results))

21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 6.8385
Test Loss: 7.10111141204834


<a name='3.3'></a>
## Cuestión 3: Añada más features a la series temporal, por ejemplo `portion_year`. Cree un modelo que mejore al anterior.


In [55]:
## Puede añadir más features
df['portion_year'] = df['Date'].dt.dayofyear / 365.0
df_multi = df[['Temp', 'portion_year']].copy()

## train - test split
train_data = df_multi.iloc[:3000].copy()
test_data = df_multi.loc[3000:, :].copy()

In [67]:
## Create windows
def create_windows_multivariate_np(data, window_size, horizon, target_col_idx, shuffle = False):

  if isinstance(data, pd.DataFrame):
    data = data.values

  x, y = [], []
  for i in range(len(data) - window_size - horizon + 1):
    x.append(data[i:i + window_size, :])
    y.append(data[i+window_size+horizon-1, target_col_idx])

  x, y = np.array(x), np.array(y)

  if shuffle:
    indices = np.arange(x.shape[0])
    np.random.shuffle(indices)
    x, y = x[indices], y[indices]
  return x, y


X_train, y_train = create_windows_multivariate_np(train_data, window_size = past, horizon = future, target_col_idx = 0)
X_test, y_test = create_windows_multivariate_np(test_data, window_size = past, horizon = future, target_col_idx = 0)

In [82]:
num_features = 1
inputs = keras.layers.Input(shape=(past, num_features))
layer1 = layers.LSTM(units = 32, return_sequences= True, name = 'layer1')(inputs)
layer2 = layers.LSTM(units = 64, return_sequences= True, name = 'layer2')(layer1)
layer3 = layers.LSTM(units = 32, return_sequences= False, name = 'layer3')(layer2)

dense = layers.Dense(1)(layer3)

model = keras.Model(inputs = inputs, outputs = dense)
model.compile(optimizer = keras.optimizers.Adam(), loss = 'mse')
model.summary()

Model: "functional_19"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_13 (InputLayer)          │ (None, 5, 1)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ layer1 (LSTM)                        │ (None, 5, 32)               │           4,352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ layer2 (LSTM)                        │ (None, 5, 64)               │          24,832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ layer3 (LSTM)                        │ (None, 32)                  │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 41,633 (162.63 KB)

 Trainable params: 41,633 (162.63 KB)

 Non-trainable params: 0 (0.00 B)

In [83]:
es_callback = keras.callbacks.EarlyStopping(
    monitor="val_loss", min_delta=0, patience=10)

history = model.fit(
    X_train, y_train,
    epochs=200,
    validation_split=0.2, shuffle=True, batch_size = 64, callbacks=[es_callback]
)

Epoch 1/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - loss: 110.9914 - val_loss: 64.8522
Epoch 2/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 80.0341 - val_loss: 33.6892
Epoch 3/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 24.6265 - val_loss: 24.1433
Epoch 4/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 18.7819 - val_loss: 19.8177
Epoch 5/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 16.9805 - val_loss: 18.1238
Epoch 6/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 16.9535 - val_loss: 17.2966
Epoch 7/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 17.1334 - val_loss: 17.0508
Epoch 8/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 16.0015 - val_loss: 16.9657
Epoch 9/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 16.3679 - val_loss: 16.9780
Epoch 10/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 16.9298 - val_loss: 17.0214
Epoch 11/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 15.5751 - val_loss: 14.8853
Epoch 12/200
38/38 ━━━━━━━━━━━━━━━━━━

In [84]:
results = model.evaluate(X_test, y_test, verbose=1)
print('Test Loss: {}'.format(results))

21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 6.7314
Test Loss: 6.991304397583008


<a name='3.4'></a>
## Cuestión 4: ¿En cuáles de estas aplicaciones se usaría un arquitectura 'many-to-one'?

**a)** Clasificación de sentimiento en textos

**b)** Verificación de voz para iniciar el ordenador.

**c)** Generación de música.

**d)** Un clasificador que clasifique piezas de música según su autor.


a), b) y d). La arquitectura Many-to-one tiene como objetivo dar una única salida partiendo de un conjunto de features. Para el caso a), la output es una salida categórica indicando el sentimiento del texto a partir de un texto con múltiples features. Para el caso b), la salida es binaria tal que por ejemplo 0 puede ser 'no iniciar el ordenador' y 1 'iniciar el ordenador' a partir de un número de features sacadas del audio. Para el último caso (d), se recibe una serie de datos devolviendo como salida una variable categórica indicando el autor. La arquitectura del caso c) es de tipo Many-to-many, es decir, se introduce un conjunto de features devolviendo una pista de música (una output con múltiples features).

<a name='3.5'></a>
## Cuestión 5: ¿Qué ventajas aporta el uso de word embeddings?

**a)** Permiten reducir la dimensión de entrada respecto al one-hot encoding.

**b)** Permiten descubrir la similaridad entre palabras de manera más intuitiva que con one-hot encoding.

**c)** Son una manera de realizar transfer learning en nlp.

**d)** Permiten visualizar las relaciones entre palabras con métodos de reducción de dimensioones como el PCA.


 a), b), c) y d).
En cuanto a la pregunta a), los word embeddings representan palabras en un espacio de menor dimensión en comparación con el one-hot encoding. Siguiendo con la pregunta b), estos permiten capturar las relaciones semánticas y sintácticas entre las palabras de manera mucho más intuitiva. A la vez, en cuanto a la pregunta c), existen embeddings pre-entrenados tal que su aplicación sea inmediata en modelos de Lenguaje Natural. Finalmente, y en relación con la pregunta a), es una ventaja la proposición expuesta en d), tal que al reducir la dimensionalidad del conjunto se accede a una nueva interpretabilidad de las relaciones entre palabras.